In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from config import g_key

In [2]:
#Pull data from Part 1(WeatherPy)
weather_data = "../WeatherPy/output_data/my_weather_df.csv"
weather_df = pd.read_csv(weather_data)
weather_df.dropna(inplace= True)
weather_df.head()

,Unnamed: 0,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,Castro,-24.7911,-50.0119,84.99,31,0,8.50,BR,1600020000
1,1,Ondjiva,-17.0667,15.7333,83.48,14,3,6.38,AO,1600020000
2,2,Bosaso,11.2842,49.1816,90.07,60,3,3.40,SO,1600020000
3,3,Hermanus,-34.4187,19.2345,56.23,73,64,29.24,ZA,1600020000
4,4,Olinda,-8.0089,-34.8553,82.04,60,37,14.52,BR,1600020000


In [3]:
#Show how many cities you"re working with 
city_count = weather_df["City"].count()
print(f"There are {city_count} cities in the dataframe.")

There are 553 cities in the dataframe.


In [4]:
#Configure gmaps with API key
gmaps.configure(api_key=g_key)

In [5]:
#Store the Latitude and longitude in variables 
locations = weather_df[["Lat", "Lng"]].astype(float)

#Store Humidity in a variable as well 
humid = weather_df["Humidity"].astype(float)

In [10]:
#Create the heat layer map 
fig = gmaps.figure(center=(48.0, 0.0), zoom_level=2)

heat_layer = gmaps.heatmap_layer(locations, weights = humid, dissipating = False,
                                 max_intensity=100, point_radius = 1)

fig.add_layer(heat_layer)

plt.savefig("output_data/heat_layer")
fig

Figure(layout=FigureLayout(height='420px'))

<Figure size 432x288 with 0 Axes>

In [30]:
#Narrow down cities to fit criteria of the temperature being between 70 and 80 degrees
#Wind speed less than 10mph, and zero cloudiness

#First find the cities between 70 and 80 degrees
less80 = weather_df.loc[weather_df["Max Temp"]<80,:]
great70 = less80.loc[weather_df["Max Temp"]>70,:]
#great70.count()
#great70.head()

#Now narrow the data more to find cities with wind speed less than 10mph
wind10 = great70.loc[great70["Wind Speed"]<10,:]
#wind10.count()
#wind10.head()

#Now reduce the list even further by finding the cities with zero cloudiness
cloud0 = wind10.loc[wind10["Cloudiness"]==0,:]
#cloud0.count()
cloud0

,Unnamed: 0,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
136,136,Fortuna,40.5982,-124.1573,70.50,72,0,6.73,US,1600020000
158,158,Angoche,-16.2325,39.9086,74.68,79,0,8.41,MZ,1600020000
313,313,Caluquembe,-13.7833,14.6833,75.52,21,0,3.60,AO,1600020000
337,337,Erzin,36.9559,36.2000,78.80,79,0,3.15,TR,1600020000
410,410,Kiryat Gat,31.6100,34.7642,79.32,58,0,9.60,IL,1600020000
482,482,Quetta,30.1990,67.0097,75.22,20,0,2.08,PK,1600020000
509,509,Semme,46.1963,1.5007,78.89,33,0,1.59,FR,1600020000
519,519,Jāwā,31.8525,35.9393,79.12,42,0,6.91,JO,1600020000
547,547,Alyangula,-13.8483,136.4192,72.34,86,0,7.29,AU,1600020000
